# 7a – Great‑circle nearest

Assign each CDC / Community Hospital to the **single acute hospital with the minimum great‑circle (haversine) distance**.  
This is a computational‑baseline mapping (no road network, no capacity).

In [ ]:

import math
import pandas as pd
from pathlib import Path

# ---------------- CONFIG --------------------------------------------------
DATA_DIR = Path('.')   # change if your CSVs live elsewhere
ACUTE_CSV = DATA_DIR / 'NHS_SW_Acute_Hospitals_enriched.csv'
CDC_CSV   = DATA_DIR / 'NHS_SW_Community_Diagnostic_Centres_enriched.csv'
CH_CSV    = DATA_DIR / 'NHS_SW_Community_Hospitals_enriched.csv'
# -------------------------------------------------------------------------

R_EARTH = 6_371.0  # km

def haversine(lat1, lon1, lat2, lon2):
    """Great‑circle distance between two (lat, lon) points in kilometres."""
    φ1, λ1, φ2, λ2 = map(math.radians, (lat1, lon1, lat2, lon2))
    dφ, dλ = φ2 - φ1, λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2 * R_EARTH * math.atan2(math.sqrt(a), math.sqrt(1 - a))

# ---------------- LOAD DATA ----------------------------------------------
acute = pd.read_csv(ACUTE_CSV)
cdc   = pd.read_csv(CDC_CSV)
ch    = pd.read_csv(CH_CSV)

# ---------------- NEAREST HUB ASSIGNMENT ---------------------------------
def label_nearest(df, hubs):
    nearest, dist_km = [], []
    for _, spoke in df.iterrows():
        dists = hubs.apply(
            lambda hub: haversine(spoke.latitude,  spoke.longitude,
                                  hub.latitude, hub.longitude),
            axis=1
        )
        idx_min = dists.idxmin()
        nearest.append(hubs.loc[idx_min, 'Name'])
        dist_km.append(dists.iloc[idx_min])
    out = df.copy()
    out['nearest_acute'] = nearest
    out['dist_km'] = dist_km
    return out

cdc   = label_nearest(cdc, acute)
ch    = label_nearest(ch,  acute)

display(cdc.head())
display(ch.head())
